In [1]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Log\data_loader.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_MP\Data\Enginered_Data\feature_enginering.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [2]:
df.head()

,Unnamed: 0,Video ID,Video Title,Channel,Published Date,Views,Likes,Comments,Subscribers,Channel Views,...,Category_Cluster_y,Year,Month,Day,DayOfWeek,DayName,Views_per_Sub,Engagement,Like_per_Sub,Comment_per_Sub
0,0,-7yvyIvJXH0,O RELÓGIO SECRETO DO GTA 5!,GamerCosta,2025-10-20 00:05:43+00:00,34727,686,16,4190000,1909505035,...,2,2025,10,20,0,Monday,0.008288,0.020214,0.000164,0.000004
1,1,9h5Uv27uTwU,"GIRLS TRIP 🌴✨ Traumurlaub, Yoga & Surfen! 🥰",Paula Doeringer,2025-10-16 16:03:51+00:00,45151,1514,21,188000,135785655,...,2,2025,10,16,3,Thursday,0.240164,0.033996,0.008053,0.000112
2,2,SmF8VTWdqZU,Imran Riaz News About TLP Saad & Anas Rizvi #r...,Real Muslim 100,2025-10-17 03:00:36+00:00,353713,6485,127,37300,54921887,...,2,2025,10,17,4,Friday,9.482668,0.018693,0.173856,0.003405
3,3,rvn2CD4h_ac,เป็นคลิปแกล้งที่ทำให้เขาเป็น Viral เลย #kornzo,Kornzo,2025-10-18 09:07:36+00:00,2171652,78431,183,43800,88085601,...,1,2025,10,18,5,Saturday,49.579964,0.036200,1.790621,0.004178
4,4,Efr59jKIkgU,โกงยันแอดมิน กลุ่มซื้อ-ขาย Valorant [ โกงมั้ยค...,DOM,2025-10-16 10:48:06+00:00,246429,9321,949,2490000,501089468,...,0,2025,10,16,3,Thursday,0.098967,0.041675,0.003743,0.000381


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8493 entries, 0 to 8492
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          8493 non-null   int64  
 1   Video ID            8493 non-null   object 
 2   Video Title         8493 non-null   object 
 3   Channel             8492 non-null   object 
 4   Published Date      8493 non-null   object 
 5   Views               8493 non-null   int64  
 6   Likes               8493 non-null   int64  
 7   Comments            8493 non-null   int64  
 8   Subscribers         8493 non-null   int64  
 9   Channel Views       8493 non-null   int64  
 10  Country             8493 non-null   object 
 11  Region              8493 non-null   object 
 12  CategoryID          8493 non-null   int64  
 13  CategoryName        8493 non-null   object 
 14  Cluster             8493 non-null   int64  
 15  CategoryID_enc      8493 non-null   int64  
 16  Countr

In [3]:
import regex as re

non_latin_pattern = re.compile(r'\P{Latin}')

def has_non_latin(s):
    if not isinstance(s, str):
        return False
    return bool(non_latin_pattern.search(s))

# faqat object turidagi ustunlarni ajratamiz
obj_cols = df.select_dtypes(include=['object']).columns

# har bir satrda qaysi ustunlarda non-latin borligini aniqlaymiz
mask = df[obj_cols].applymap(has_non_latin).any(axis=1)
notanish_df = df[mask]

print(f"Notanish yozuvli satrlar soni: {mask.sum()}")
notanish_df.to_csv("nonlatin_rows.csv", index=False)


Notanish yozuvli satrlar soni: 8493


C:\Users\Rasulbek907\AppData\Local\Temp\ipykernel_1612\598365124.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = df[obj_cols].applymap(has_non_latin).any(axis=1)


In [4]:
summary = df.select_dtypes(include=['object']).apply(
    lambda col: col.apply(has_non_latin).sum()
)
print(summary)


Video ID          7612
Video Title       8462
Channel           6627
Published Date    8493
Country           1649
Region               0
CategoryName      4714
Anomaly              0
DayName              0
dtype: int64


In [7]:
df["CategoryName"].unique()

array(['Gaming', 'People & Blogs', 'News & Politics', 'Entertainment',
       'Science & Technology', 'Howto & Style', 'Autos & Vehicles',
       'Music', 'Film & Animation', 'Sports', 'Comedy'], dtype=object)

In [5]:
category_map = {
    'Gaming': 0,
    'People & Blogs': 1,
    'News & Politics': 2,
    'Entertainment': 3,
    'Science & Technology': 4,
    'Howto & Style': 5,
    'Autos & Vehicles': 6,
    'Music': 7,
    'Film & Animation': 8,
    'Sports': 9,
    'Comedy': 10
}

df['CategoryName'] = df['CategoryName'].map(category_map)


In [6]:
summary = df.select_dtypes(include=['object']).apply(
    lambda col: col.apply(has_non_latin).sum()
)
print(summary)

Video ID          7612
Video Title       8462
Channel           6627
Published Date    8493
Country           1649
Region               0
Anomaly              0
DayName              0
dtype: int64


In [7]:
from unidecode import unidecode

# notanish yozuvli ustunlar ro'yxati
cols_to_clean = ['Video ID', 'Video Title', 'Channel', 'Published Date', 'Country']

# har bir ustun bo'yicha unidecode qo'llash
for col in cols_to_clean:
    df[col] = df[col].astype(str).apply(unidecode)

print("✅ Unidecode barcha belgilar lotin yozuviga o‘tkazildi!")


✅ Unidecode barcha belgilar lotin yozuviga o‘tkazildi!


In [10]:
df.head()

,Unnamed: 0,Video ID,Video Title,Channel,Published Date,Views,Likes,Comments,Subscribers,Channel Views,...,Category_Cluster_y,Year,Month,Day,DayOfWeek,DayName,Views_per_Sub,Engagement,Like_per_Sub,Comment_per_Sub
0,0,-7yvyIvJXH0,O RELOGIO SECRETO DO GTA 5!,GamerCosta,2025-10-20 00:05:43+00:00,34727,686,16,4190000,1909505035,...,2,2025,10,20,0,Monday,0.008288,0.020214,0.000164,0.000004
1,1,9h5Uv27uTwU,"GIRLS TRIP Traumurlaub, Yoga & Surfen!",Paula Doeringer,2025-10-16 16:03:51+00:00,45151,1514,21,188000,135785655,...,2,2025,10,16,3,Thursday,0.240164,0.033996,0.008053,0.000112
2,2,SmF8VTWdqZU,Imran Riaz News About TLP Saad & Anas Rizvi #r...,Real Muslim 100,2025-10-17 03:00:36+00:00,353713,6485,127,37300,54921887,...,2,2025,10,17,4,Friday,9.482668,0.018693,0.173856,0.003405
3,3,rvn2CD4h_ac,epnkhlipaeklngthiithamaihekhaaepn Viral ely #k...,Kornzo,2025-10-18 09:07:36+00:00,2171652,78431,183,43800,88085601,...,1,2025,10,18,5,Saturday,49.579964,0.036200,1.790621,0.004178
4,4,Efr59jKIkgU,okngyanae`dmin klumchuue`-khaay Valorant [ okn...,DOM,2025-10-16 10:48:06+00:00,246429,9321,949,2490000,501089468,...,0,2025,10,16,3,Thursday,0.098967,0.041675,0.003743,0.000381


In [11]:
import regex as re

# faqat lotin harflar, raqamlar, bo‘sh joy va oddiy belgilarni ruxsat beramiz
normal_pattern = re.compile(r'^[\p{Latin}\d\s.,!?;:\-_/()\'"&%]*$')

def has_unusual_chars(s):
    if not isinstance(s, str):
        return False
    return not bool(normal_pattern.match(s))  # agar mos kelmasa → noodatiy belgi bor

# tekshiriladigan ustunlar
cols_to_check = ['Video ID', 'Video Title', 'Channel', 'Published Date', 'Country']

# har bir ustun bo‘yicha noodatiy belgilar sonini hisoblash
for col in cols_to_check:
    count = df[col].apply(has_unusual_chars).sum()
    print(f"{col:20s} → {count} ta noodatiy qiymat")


Video ID             → 0 ta noodatiy qiymat
Video Title          → 4240 ta noodatiy qiymat
Channel              → 354 ta noodatiy qiymat
Published Date       → 8493 ta noodatiy qiymat
Country              → 0 ta noodatiy qiymat


# Video Title va Channel ustunlaridagi noodatiy belgilarni tozalash

In [12]:
import re

def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'[^\x00-\x7F]+', '', text)  # faqat ASCII qoldiradi
    return text

df["Video Title"] = df["Video Title"].apply(clean_text)
df["Channel"] = df["Channel"].apply(clean_text)


# Published Date ustunini datetime formatga o‘tkazish

In [13]:
df["Published Date"] = pd.to_datetime(df["Published Date"], errors='coerce')


In [14]:
import re
import unicodedata

def deep_clean_text(text):
    if not isinstance(text, str):
        return text

    # 1. Normalize (unicode form)
    text = unicodedata.normalize('NFKC', text)

    # 2. Remove control characters va ko‘rinmas belgilar
    text = re.sub(r'[\u200b-\u200f\u202a-\u202e\u2060-\u206f\u00ad\uFEFF]', '', text)

    # 3. Remove HTML entities
    text = re.sub(r'&[a-z]+;', '', text)

    # 4. Remove all non-printable characters
    text = ''.join(ch for ch in text if ch.isprintable())

    # 5. Remove remaining non-ASCII characters (optional)
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # 6. Strip whitespaces
    text = text.strip()

    return text

df["Video Title"] = df["Video Title"].apply(deep_clean_text)
df["Channel"] = df["Channel"].apply(deep_clean_text)


In [15]:
def count_unusual(text):
    if not isinstance(text, str):
        return False
    # Agar matnda ASCII’dan tashqari belgilar bo‘lsa, True
    return bool(re.search(r'[^\x00-\x7F]', text))

for col in ["Video Title", "Channel"]:
    bad_count = df[col].apply(count_unusual).sum()
    print(f"{col:20} → {bad_count} ta noodatiy qiymat")


Video Title          → 0 ta noodatiy qiymat
Channel              → 0 ta noodatiy qiymat


# DF dagi noodatiy belgilarni tekshirish 

In [16]:
import re

def count_unusual_chars(df):
    pattern = re.compile(r'[^\x00-\x7F]')  # ASCII bo'lmagan belgilarni topadi
    result = {}

    for col in df.select_dtypes(include=['object']).columns:
        # Har bir ustun uchun noodatiy belgilar bor-yo‘qligini tekshirish
        bad_count = df[col].astype(str).apply(lambda x: bool(pattern.search(x))).sum()
        result[col] = bad_count

    return result


unusual_counts = count_unusual_chars(df)

for col, count in unusual_counts.items():
    print(f"{col:20} → {count} ta noodatiy qiymat")


Video ID             → 0 ta noodatiy qiymat
Video Title          → 0 ta noodatiy qiymat
Channel              → 0 ta noodatiy qiymat
Country              → 0 ta noodatiy qiymat
Region               → 0 ta noodatiy qiymat
Anomaly              → 0 ta noodatiy qiymat
DayName              → 0 ta noodatiy qiymat


In [17]:
import sys
import os

# Source papkasini yo'lga qo'shamiz
source_path = os.path.abspath("../Source")
if source_path not in sys.path:
    sys.path.append(source_path)

print("Python executable:", sys.executable)
print("Source path added:", source_path)

# Endi modulni import qilamiz
from preprosessing import Cleaner, Encoder, Scaler

Python executable: c:\Users\Rasulbek907\Desktop\Project_MP\mpvenv\Scripts\python.exe
Source path added: c:\Users\Rasulbek907\Desktop\Project_MP\Source


In [18]:
cleaner = Cleaner(df)
df = cleaner.tozala().get_df()

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8493 entries, 0 to 8492
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   Unnamed: 0          8493 non-null   int64              
 1   Video ID            8493 non-null   object             
 2   Video Title         8493 non-null   object             
 3   Channel             8493 non-null   object             
 4   Published Date      8493 non-null   datetime64[ns, UTC]
 5   Views               8493 non-null   int64              
 6   Likes               8493 non-null   int64              
 7   Comments            8493 non-null   int64              
 8   Subscribers         8493 non-null   int64              
 9   Channel Views       8493 non-null   int64              
 10  Country             8493 non-null   object             
 11  Region              8493 non-null   object             
 12  CategoryID          8493 non-null 

In [20]:
encoder = Encoder(df, target_col="CategoryName")
encoded_df = encoder.encodla().get_df()

In [21]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8493 entries, 0 to 8492
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   Unnamed: 0          8493 non-null   int64              
 1   Video ID            8493 non-null   int64              
 2   Video Title         8493 non-null   int64              
 3   Channel             8493 non-null   int64              
 4   Published Date      8493 non-null   datetime64[ns, UTC]
 5   Views               8493 non-null   int64              
 6   Likes               8493 non-null   int64              
 7   Comments            8493 non-null   int64              
 8   Subscribers         8493 non-null   int64              
 9   Channel Views       8493 non-null   int64              
 10  Country             8493 non-null   int64              
 11  Region              8493 non-null   int64              
 12  CategoryID          8493 non-null 

In [22]:
scaler = Scaler(df)
df = scaler.scaling_qil().get_df()

In [23]:

df.head()

,Unnamed: 0,Video ID,Video Title,Channel,Published Date,Views,Likes,Comments,Subscribers,Channel Views,...,Category_Cluster_y,Year,Month,Day,DayOfWeek,DayName,Views_per_Sub,Engagement,Like_per_Sub,Comment_per_Sub
0,-1.731847,-1.721242,0.094898,-0.733313,2025-10-20 00:05:43+00:00,-0.378194,-0.362793,-0.100989,0.183348,0.021778,...,0.596018,0.0,0.141625,1.225846,-1.858123,Monday,0.008288,0.020214,0.000164,0.000004
1,-1.731439,-1.141648,-0.732741,0.326218,2025-10-16 16:03:51+00:00,-0.377046,-0.356911,-0.100484,-0.207709,-0.161989,...,0.596018,0.0,0.141625,0.113043,-0.287141,Thursday,0.240164,0.033996,0.008053,0.000112
2,-1.731031,-0.104824,-0.486586,0.469163,2025-10-17 03:00:36+00:00,-0.343066,-0.321594,-0.089758,-0.222435,-0.170367,...,0.596018,0.0,0.141625,0.391244,0.236520,Friday,9.482668,0.018693,0.173856,0.003405
3,-1.730623,1.301536,1.291973,-0.315894,2025-10-18 09:07:36+00:00,-0.142867,0.189549,-0.084092,-0.221800,-0.166931,...,-0.298799,0.0,0.141625,0.669444,0.760181,Saturday,49.579964,0.036200,1.790621,0.004178
4,-1.730215,-0.865923,1.563195,-1.139314,2025-10-16 10:48:06+00:00,-0.354881,-0.301446,-0.006584,0.017232,-0.124141,...,-1.193616,0.0,0.141625,0.113043,-0.287141,Thursday,0.098967,0.041675,0.003743,0.000381


# MinMaxScaler bilan datetime ustunini scaling qilish

In [26]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# (1) Ustun datetime formatda ekanligiga ishonch hosil qilamiz
df['Published Date'] = pd.to_datetime(df['Published Date'], utc=True)

# (2) Datetime ni soniyalarga (timestamp) aylantirish
df['Published Date'] = df['Published Date'].astype('int64') / 1e9  # nanosekund → sekund

# (3) NaN qiymatlarni hisobdan chiqaramiz
mask = df['Published Date'].notna()

# (4) Scaling
scaler = MinMaxScaler(feature_range=(0, 1))
df.loc[mask, 'Published Date'] = scaler.fit_transform(df.loc[mask, ['Published Date']])

# (5) Natijani ko‘rish
print(df['Published Date'].head())


0    0.988754
1    0.898341
2    0.910707
3    0.944730
4    0.892396
Name: Published Date, dtype: float64


In [27]:
df.head()

,Unnamed: 0,Video ID,Video Title,Channel,Published Date,Views,Likes,Comments,Subscribers,Channel Views,...,Month,Day,DayOfWeek,DayName,Views_per_Sub,Engagement,Like_per_Sub,Comment_per_Sub,Published_ts,Published_scaled
0,-1.731847,-1.721242,0.094898,-0.733313,0.988754,-0.378194,-0.362793,-0.100989,0.183348,0.021778,...,0.141625,1.225846,-1.858123,Monday,0.008288,0.020214,0.000164,0.000004,1.760919e+09,0.988754
1,-1.731439,-1.141648,-0.732741,0.326218,0.898341,-0.377046,-0.356911,-0.100484,-0.207709,-0.161989,...,0.141625,0.113043,-0.287141,Thursday,0.240164,0.033996,0.008053,0.000112,1.760631e+09,0.898341
2,-1.731031,-0.104824,-0.486586,0.469163,0.910707,-0.343066,-0.321594,-0.089758,-0.222435,-0.170367,...,0.141625,0.391244,0.236520,Friday,9.482668,0.018693,0.173856,0.003405,1.760670e+09,0.910707
3,-1.730623,1.301536,1.291973,-0.315894,0.944730,-0.142867,0.189549,-0.084092,-0.221800,-0.166931,...,0.141625,0.669444,0.760181,Saturday,49.579964,0.036200,1.790621,0.004178,1.760778e+09,0.944730
4,-1.730215,-0.865923,1.563195,-1.139314,0.892396,-0.354881,-0.301446,-0.006584,0.017232,-0.124141,...,0.141625,0.113043,-0.287141,Thursday,0.098967,0.041675,0.003743,0.000381,1.760612e+09,0.892396


# DayName ustunini avval Label Encoding, so‘ngra MinMax Scaling qilish 

In [28]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# (1) Misol DataFrame
# Sizning holatingizda df allaqachon mavjud bo‘ladi
# df = pd.read_csv("file.csv")
# Masalan:
# df = pd.DataFrame({"DayName": ["Monday", "Thursday", "Tuesday", "Friday", "Monday"]})

# (2) NaN qiymatlarni hisobga olish uchun tozalaymiz
mask = df['DayName'].notna()

# (3) Label Encoding
le = LabelEncoder()
df.loc[mask, 'DayName'] = le.fit_transform(df.loc[mask, 'DayName'])

# (4) Label encodingdan keyin Min-Max Scaling
scaler = MinMaxScaler(feature_range=(0, 1))
df.loc[mask, 'DayName'] = scaler.fit_transform(df.loc[mask, ['DayName']])

# (5) Natijani ko‘rish
print(df.head())


   Unnamed: 0  Video ID  Video Title   Channel  Published Date     Views  \
0   -1.731847 -1.721242     0.094898 -0.733313        0.988754 -0.378194   
1   -1.731439 -1.141648    -0.732741  0.326218        0.898341 -0.377046   
2   -1.731031 -0.104824    -0.486586  0.469163        0.910707 -0.343066   
3   -1.730623  1.301536     1.291973 -0.315894        0.944730 -0.142867   
4   -1.730215 -0.865923     1.563195 -1.139314        0.892396 -0.354881   

      Likes  Comments  Subscribers  Channel Views  ...     Month       Day  \
0 -0.362793 -0.100989     0.183348       0.021778  ...  0.141625  1.225846   
1 -0.356911 -0.100484    -0.207709      -0.161989  ...  0.141625  0.113043   
2 -0.321594 -0.089758    -0.222435      -0.170367  ...  0.141625  0.391244   
3  0.189549 -0.084092    -0.221800      -0.166931  ...  0.141625  0.669444   
4 -0.301446 -0.006584     0.017232      -0.124141  ...  0.141625  0.113043   

   DayOfWeek   DayName  Views_per_Sub  Engagement  Like_per_Sub  \
0  -1.8

In [29]:
df.head()

,Unnamed: 0,Video ID,Video Title,Channel,Published Date,Views,Likes,Comments,Subscribers,Channel Views,...,Month,Day,DayOfWeek,DayName,Views_per_Sub,Engagement,Like_per_Sub,Comment_per_Sub,Published_ts,Published_scaled
0,-1.731847,-1.721242,0.094898,-0.733313,0.988754,-0.378194,-0.362793,-0.100989,0.183348,0.021778,...,0.141625,1.225846,-1.858123,0.166667,0.008288,0.020214,0.000164,0.000004,1.760919e+09,0.988754
1,-1.731439,-1.141648,-0.732741,0.326218,0.898341,-0.377046,-0.356911,-0.100484,-0.207709,-0.161989,...,0.141625,0.113043,-0.287141,0.666667,0.240164,0.033996,0.008053,0.000112,1.760631e+09,0.898341
2,-1.731031,-0.104824,-0.486586,0.469163,0.910707,-0.343066,-0.321594,-0.089758,-0.222435,-0.170367,...,0.141625,0.391244,0.236520,0.0,9.482668,0.018693,0.173856,0.003405,1.760670e+09,0.910707
3,-1.730623,1.301536,1.291973,-0.315894,0.944730,-0.142867,0.189549,-0.084092,-0.221800,-0.166931,...,0.141625,0.669444,0.760181,0.333333,49.579964,0.036200,1.790621,0.004178,1.760778e+09,0.944730
4,-1.730215,-0.865923,1.563195,-1.139314,0.892396,-0.354881,-0.301446,-0.006584,0.017232,-0.124141,...,0.141625,0.113043,-0.287141,0.666667,0.098967,0.041675,0.003743,0.000381,1.760612e+09,0.892396


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8493 entries, 0 to 8492
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          8493 non-null   float64
 1   Video ID            8493 non-null   float64
 2   Video Title         8493 non-null   float64
 3   Channel             8493 non-null   float64
 4   Published Date      8493 non-null   float64
 5   Views               8493 non-null   float64
 6   Likes               8493 non-null   float64
 7   Comments            8493 non-null   float64
 8   Subscribers         8493 non-null   float64
 9   Channel Views       8493 non-null   float64
 10  Country             8493 non-null   float64
 11  Region              8493 non-null   float64
 12  CategoryID          8493 non-null   float64
 13  CategoryName        8493 non-null   float64
 14  Cluster             8493 non-null   float64
 15  CategoryID_enc      8493 non-null   float64
 16  Countr

In [31]:
print(df.columns.tolist())


['Unnamed: 0', 'Video ID', 'Video Title', 'Channel', 'Published Date', 'Views', 'Likes', 'Comments', 'Subscribers', 'Channel Views', 'Country', 'Region', 'CategoryID', 'CategoryName', 'Cluster', 'CategoryID_enc', 'Country_enc', 'VideoCluster', 'Anomaly', 'Channel_Cluster', 'Video_Cluster', 'Country_Cluster_x', 'Category_Cluster_x', 'Country_Cluster_y', 'Category_Cluster_y', 'Year', 'Month', 'Day', 'DayOfWeek', 'DayName', 'Views_per_Sub', 'Engagement', 'Like_per_Sub', 'Comment_per_Sub', 'Published_ts', 'Published_scaled']


In [32]:
# Belgilangan ustunlarni o‘chirish
df = df.drop(columns=['Unnamed: 0', 'Anomaly', 'Published_scaled'], errors='ignore')

# Natijani tekshirish
print(df.columns.tolist())


['Video ID', 'Video Title', 'Channel', 'Published Date', 'Views', 'Likes', 'Comments', 'Subscribers', 'Channel Views', 'Country', 'Region', 'CategoryID', 'CategoryName', 'Cluster', 'CategoryID_enc', 'Country_enc', 'VideoCluster', 'Channel_Cluster', 'Video_Cluster', 'Country_Cluster_x', 'Category_Cluster_x', 'Country_Cluster_y', 'Category_Cluster_y', 'Year', 'Month', 'Day', 'DayOfWeek', 'DayName', 'Views_per_Sub', 'Engagement', 'Like_per_Sub', 'Comment_per_Sub', 'Published_ts']


In [34]:
pd.set_option('display.max_columns', None)  # barcha ustunlarni ko‘rsatish
pd.set_option('display.max_rows', 50)       # 50 qatordan ko‘p chiqmasin
display(df[cols])


,Video ID,Video Title,Channel,Published Date,Views,Likes,Comments,Subscribers,Channel Views,Country,Region,CategoryID,CategoryName,Cluster,CategoryID_enc,Country_enc,VideoCluster,Channel_Cluster,Video_Cluster,Country_Cluster_x,Category_Cluster_x,Country_Cluster_y,Category_Cluster_y,Year,Month,Day,DayOfWeek,DayName,Views_per_Sub,Engagement,Like_per_Sub,Comment_per_Sub,Published_ts
0,-1.721242,0.094898,-0.733313,0.988754,-0.378194,-0.362793,-0.100989,0.183348,0.021778,-1.717384,-1.393303,0.248318,-1.519788,-0.176027,-0.361945,-1.717384,-1.190758,-0.227665,-0.384714,-0.309081,0.596018,-0.309081,0.596018,0.0,0.141625,1.225846,-1.858123,0.166667,0.008288,0.020214,0.000164,0.000004,1.760919e+09
1,-1.141648,-0.732741,0.326218,0.898341,-0.377046,-0.356911,-0.100484,-0.207709,-0.161989,-1.331542,-0.978419,0.465648,-1.205850,-0.176027,0.988521,-1.331542,-1.190758,-0.227665,-0.384714,-0.309081,0.596018,-0.309081,0.596018,0.0,0.141625,0.113043,-0.287141,0.666667,0.240164,0.033996,0.008053,0.000112,1.760631e+09
2,-0.104824,-0.486586,0.469163,0.910707,-0.343066,-0.321594,-0.089758,-0.222435,-0.170367,0.597666,0.681117,0.791642,-0.891912,-0.176027,0.650904,0.597666,-0.111105,-0.227665,-0.384714,-0.309081,0.596018,-0.309081,0.596018,0.0,0.141625,0.391244,0.236520,0.0,9.482668,0.018693,0.173856,0.003405,1.760670e+09
3,1.301536,1.291973,-0.315894,0.944730,-0.142867,0.189549,-0.084092,-0.221800,-0.166931,1.099260,1.234296,0.682978,-0.577974,-0.176027,-1.037178,1.099260,0.968548,-0.227665,-0.384714,-0.309081,-0.298799,-0.309081,-0.298799,0.0,0.141625,0.669444,0.760181,0.333333,49.579964,0.036200,1.790621,0.004178,1.760778e+09
4,-0.865923,1.563195,-1.139314,0.892396,-0.354881,-0.301446,-0.006584,0.017232,-0.124141,1.099260,1.234296,1.117637,-0.264036,-0.176027,1.326137,1.099260,-0.111105,-0.227665,-0.384714,-0.309081,-1.193616,-0.309081,-1.193616,0.0,0.141625,0.113043,-0.287141,0.666667,0.098967,0.041675,0.003743,0.000381,1.760612e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8488,0.701957,-0.900405,-0.478934,0.951389,-0.227447,0.141167,0.559342,1.913888,0.835670,0.404745,-1.393303,0.248318,-1.519788,-0.176027,-0.361945,0.404745,0.968548,4.045387,-0.384714,-0.309081,0.596018,-0.309081,0.596018,0.0,0.141625,0.669444,0.760181,0.333333,0.064092,0.055687,0.003270,0.000299,1.760800e+09
8489,0.431534,-1.434219,-0.413170,0.838615,-0.358168,-0.332904,-0.099775,-0.151327,-0.013382,-0.598443,-0.356093,0.791642,-0.891912,-0.176027,0.650904,-0.598443,-1.190758,-0.227665,-0.384714,1.595871,0.596018,1.595871,0.596018,0.0,0.141625,-0.443359,-1.334463,0.833333,0.283115,0.022721,0.006396,0.000037,1.760440e+09
8490,-0.406654,-1.286691,0.828581,0.869016,-0.275437,-0.306170,-0.095829,-0.144585,-0.000324,1.407934,1.580032,-0.077676,1.305654,-0.176027,1.663754,1.407934,-0.111105,-0.227665,-0.384714,-0.309081,-1.193616,-0.309081,-1.193616,0.0,0.141625,-0.165158,-0.810802,1.0,1.160466,0.009013,0.010379,0.000080,1.760537e+09
8491,-1.659245,-1.602706,0.161808,0.915131,-0.275500,-0.280104,-0.083788,0.975820,0.372619,-0.559859,-0.286946,0.791642,-0.891912,-0.176027,0.650904,-0.559859,-1.190758,-0.227665,-0.384714,-2.214032,0.596018,-2.214032,0.596018,0.0,0.141625,0.391244,0.236520,0.0,0.078639,0.012934,0.001002,0.000015,1.760684e+09


In [35]:

df.to_csv(r"C:\Users\Rasulbek907\Desktop\Project_MP\Data\Preprosessed\Preprosessed.csv")